In [126]:
# encoding=utf-8
import re
import jieba_fast as jieba
import wordcloud
import urllib
import json
import pandas as pd
import pyecharts
from pyecharts.charts import Bar
from pyecharts.charts import Line
from pyecharts.charts import Pie
from pyecharts.charts import WordCloud
from matplotlib import pyplot
import matplotlib.pyplot as plt
from pandas import DataFrame,Series
# from xpinyin import Pinyin 
# from Pinyin2Hanzi import DefaultDagParams
# from Pinyin2Hanzi import dag

# 解决乱码问题
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

In [127]:
data = pd.read_csv("data/job.csv",encoding='gbk')
data = data.fillna(0)
# 前几条数据
data.head()

,_id,url,title,money,company,experience,requirement,detail,goods
0,5f0112d93342821665175723,https://jobs.51job.com/shanghai-hkq/116974871....,k12班主任 学管师 欢迎优秀应届生,1-1.5万/月,清羽教育,招若干人,07-04发布,【岗位职责】1.了解学生的学习进展情况并向家长沟通反馈，对所带在读学员提供高水平的服务，促使...,带薪年假、五险一金、绩效奖金、节日福利、专业培训、加班补贴、补充医疗保险、员工旅游、定期体检...
1,5f0112d93342821665175724,https://jobs.51job.com/guangzhou-pyq/116429800...,高级需求分析师,0.8-1.2万/月,广东中象集运科技有限公司,3-4年经验,本科,岗位职责：1、协助产品经理关于平台建设及产品的调研、分析、规划设计2、负责产品功能设计，绘制...,五险一金、专业培训、绩效奖金、年终奖金、员工旅游
2,5f0112d93342821665175725,https://jobs.51job.com/shanghai-pdxq/116409796...,H5前端,1-1.8万/月,上海狂龙数字科技股份有限公司,3-4年经验,本科,工作职责:1、负责前端5核心代码的开发，解决前端的各种问题，前端工程搭建、框架整合、界面优化...,五险一金、绩效奖金、做五休二、带薪年假、周末双休、节日福利
3,5f0112d93342821665175726,https://jobs.51job.com/shenzhen-ftq/116959378....,外贸业务员,6-8千/月,深圳市勃维科技有限公司,大专,招2人,任职要求：1. 大专以上学历22 -35岁，具有与客户通过和电话流畅交流的能力2. 熟悉外贸...,绩效奖金、带薪年假、年底双薪、五险一金、餐饮补贴
4,5f0112d93342821665175727,https://jobs.51job.com/shanghai-pdxq/116423119...,网易产品销售（5k+不加班）,0.5-1万/月,上海四易信息科技有限公司,招3人,07-04发布,1、对上海企业部负责人或法人进行网易企业邮箱:..产品的推广销售2、寻找***，利用电话、网...,做五休二、交通补贴、带薪年假、五险一金、节日福利、全勤奖、绩效奖金


In [11]:
# 教学经历
data_requirement = data['requirement'].value_counts()
filter_requirement = ['大专','本科','中专','高中','初中及以下','中技','硕士','博士']
data_tmp = []
data_value = []
for (key,value) in data_requirement.items():
    if key not in filter_requirement:
        del data_requirement[key]
    else:
        data_tmp.append(key)
        data_value.append(value)
# plt.title('教学经历')
# # 显示数值
# i = 0
# for (key,value) in data_requirement.items():
#     plt.text(i,value,value)
#     i += 1
# plt.show()
bar = Bar()
bar.add_xaxis(data_tmp)
bar.add_yaxis("教学经历", data_value)
bar.render("教学经历.html")

D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\教学经历.html'

In [14]:
# 工作经验
data_experience = data['experience'].value_counts()
filter_experience = ['1年经验','无需经验','3-4年经验','2年经验','在校生/应届生','5-7年经验','8-9年经验','10年以上经验']
data_tmp = []
data_value = []
for (key,value) in data_experience.items():
    if key not in filter_experience:
        del data_experience[key]
        continue
#     data_experience[re.sub('经验','',key)] = data_experience.pop(key) # 去掉"经验"
    data_tmp.append(re.sub('经验','',key))
    data_value.append(value)
# data_experience.plot(kind='barh',rot=0)
# plt.title('工作经验')
# # 显示数值
# i = 0
# for (key,value) in data_experience.items():
#     plt.text(value,i,value)
#     i += 1
# plt.show()
bar = Bar()
bar.add_xaxis(data_tmp)
bar.add_yaxis("工作经验", data_value)
bar.render("工作经验.html")

D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\工作经验.html'

In [142]:
# 职位
final = ''
stopwords = ['编号','代表','五险','一金']
data_final = {}
for n in range(len(data['title'])):
    seg_list = list(jieba.cut(data['title'][n]))
    for seg in seg_list:
        if seg not in stopwords:
#             final += seg + ' '
            data_final.setdefault(seg,0)
            data_final[seg] += 1
data_final = sorted(data_final.items(), key= lambda item:item[1])
word = WordCloud()
word.add("职位",data_final,word_size_range=(20,100))
word.render("职位词云.html")
# my_wordcloud = wordcloud.WordCloud(font_path = r'C:\Windows\Fonts\simsun.ttc',background_color=None,mode='RGBA').generate(final)
# plt.imshow(my_wordcloud)
# plt.axis('off')
# plt.show()

D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\职位词云.html'

In [141]:
# 福利
final = ''
stopwords = []
data_final = {}
for n in range(len(data['goods'])):
    seg_list = list(jieba.cut(str(data['goods'][n])))
    for seg in seg_list:
        if seg not in stopwords:
#             final += seg + ' '
            data_final.setdefault(seg,0)
            data_final[seg] += 1
data_final = sorted(data_final.items(), key= lambda item:item[1])
word = WordCloud()
word.add("福利",data_final,word_size_range=(20,100))
word.render("福利词云.html")
# my_wordcloud = wordcloud.WordCloud(font_path = r'C:\Windows\Fonts\simsun.ttc',background_color=None,mode='RGBA').generate(final)
# plt.imshow(my_wordcloud)
# plt.axis('off')
# plt.savefig("福利.png")
# plt.show()

D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\福利词云.html'

In [140]:
# 详细工作
final = ''
stopwords = ['职能','类别','要求','微信','分享','任职']
data_final = {}
for n in range(1,10000):
    seg_list = list(jieba.cut(str(data['detail'][n])))
    for seg in seg_list:
        if seg not in stopwords:
#             final += seg + ' '
            data_final.setdefault(seg,0)
            data_final[seg] += 1
data_final = sorted(data_final.items(), key= lambda item:item[1])
word = WordCloud()
word.add("详细工作",data_final,shape='circle',word_size_range=(20,100))
word.render("详细工作词云.html")
# my_wordcloud = wordcloud.WordCloud(font_path = r'C:\Windows\Fonts\simsun.ttc',background_color=None,mode='RGBA').generate(final)
# plt.imshow(my_wordcloud)
# plt.axis('off')
# plt.savefig("详细工作.png")
# plt.show()

D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\详细工作词云.html'

In [6]:
data_company = data['company'].value_counts()
# data_company['其他'] = 0 
data_tmp = []
data_value = []
for (key,value) in data_company.items():
    if value < 500:
#         data_company['其他'] += value
        del data_company[key]
        continue
#     data_company[re.sub(r'有限公司','',key)] = data_company[key]
    data_tmp.append(re.sub(r'有限公司','',key))
    data_value.append(value)
# data_company.plot(kind='barh',rot=0)
# plt.title('公司')
# # 显示数值
# i = 0
# for (key,value) in data_company.items():
#     plt.text(value,i,value)
#     i += 1
# plt.show()
bar = Bar()
bar.add_xaxis(data_tmp)
bar.add_yaxis("公司", data_value)
bar.render("公司.html")

D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\公司.html'

In [42]:
# for (key,value) in enumerate(data['money']):
#         num = re.search(r'([\d|\.]+)',str(value))
#         uunit = re.search(r'([万|千])',str(value))
#         if num:
#             num = num.group()
#         if uunit:
#             uunit = uunit.group()
#         data['money'][str(key)] = str(num) + str(uunit)
data_tmp = []
data_value = []
data_money = data['money'].str.extract('([\d|\.].*?)')
# data_money = {}
data_money = data['money'].value_counts()
for (key,value) in data_money.items():
    if value < 2000:
        del data_money[key]
        continue
    data_tmp.append(key)
    data_value.append(value)
# #     data_money[re.sub(r'/月','',str(key))] = data_money[str(key)]
# data_money.plot(kind='barh',rot=0)
# plt.title('工资区间/月')
# plt.show()

bar = Line()
bar.add_xaxis(data_tmp)
bar.add_yaxis("工资区间", data_value)
bar.render("工资区间.html")

<>:11: DeprecationWarning: invalid escape sequence \d
<>:11: DeprecationWarning: invalid escape sequence \d
<>:11: DeprecationWarning: invalid escape sequence \d
<ipython-input-42-d44516958d20>:11: DeprecationWarning: invalid escape sequence \d
  data_money = data['money'].str.extract('([\d|\.].*?)')
D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\工资区间.html'

In [41]:
data_money = data['money'].str.extract('-([\d|\.|万|千]+)')
data_tmp = []
data_value = []
# data_money = {}
# for value in data['money'].items():
#     if key < 
data_money = data_money[0].value_counts()
for (key,value) in data_money.items():
    if value < 1000:
        del data_money[key]
        continue
    data_tmp.append(key)
    data_value.append(value)
# data_money.plot(kind='barh',rot=0)
# plt.title('最高工资频次')
# plt.show()

bar = Line()
bar.add_xaxis(data_tmp)
bar.add_yaxis("最高工资", data_value)
bar.render("最高工资.html")

<>:1: DeprecationWarning: invalid escape sequence \d
<>:1: DeprecationWarning: invalid escape sequence \d
<>:1: DeprecationWarning: invalid escape sequence \d
<ipython-input-41-6d21421a5edb>:1: DeprecationWarning: invalid escape sequence \d
  data_money = data['money'].str.extract('-([\d|\.|万|千]+)')
D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\最高工资.html'

In [102]:
def to_chinese(text):
    """
    英文转中文
    """
    url = "http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&sessionFrom=null"

    headers = {
        "Accept":"application/json, text/javascript, */*; q=0.01",
        "X-Requested-With":"XMLHttpRequest",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 ",
        "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8"
        }

    data = {
        "i":text,
        "to":"AUTO",
        "from":"AUTO",
        "smartresult":"dict",
        "client":"fanyideskweb",
        "salt": "1508029833145",
        "sign": "7df48510eed000cc4782838d7d1a55e6",
        "doctype":"json",
        "version":"2.1",
        "keyfrom":"fanyi.web",
        "action":"FY_BY_CLICKBUTTION",
        "typoResult":"true"
        }

    data = urllib.parse.urlencode(data).encode(encoding='utf-8')

    request = urllib.request.Request(url, data = data, headers = headers)
    response = urllib.request.urlopen(request)
    return json.loads(response.read())['translateResult'][0][0]['tgt']

In [108]:
# https://jobs.51job.com/shanghai-hkq/116974871....
data_address = data['url'].str.extract(r'https://[a-z|.].*?/([a-z].*?)[\-|/].*?')
data_tmp = []
data_value = []
# data_money = {}
# for value in data['money'].items():
#     if key < 
data_address = data_address[0].value_counts()
for (key,value) in data_address.items():
    if value < 3000:
        del data_address[key]
        continue
    data_tmp.append(key)
    data_value.append(value)
#     # 拼音转汉字
#     translator=DefaultDagParams()
#     result = dag(translator, key ,path_num=5,log=True)
#     data_address[result] = data_address.pop(key)
# data_address.plot(kind='barh',rot=0)
# plt.title('城市频次')
# plt.show()
pie = Pie()
pie.add('城市', [(to_chinese(key), value) for (key ,value) in data_address.items()] , color=None)
# pie.show_config()
pie.render('城市.html')

D:\Anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'F:\\Python\\zhihao\\view\\城市.html'